## Graphviz

Let's play around with the graphviz backend.

### Shift register

This example is taken from the shift_register notebook.

In [1]:
from magma import *
from mantle import DefineRegister

N = 4
Register4 = DefineRegister(4)
T = Bits(N)

class ShiftRegister(Circuit):
    name = "ShiftRegister"
    IO = ["I", In(T), "O", Out(T), "CLK", In(Clock)]
    @classmethod
    def definition(io):
        regs = [Register4() for _ in range(N)]
        [wire(io.CLK, reg.CLK) for reg in regs]
        wire(io.I, getattr(regs[0], "I"))
        braid(regs, foldargs={"I":"O"})
        wire(regs[-1].O, io.O)


Now let's visualize our `ShiftRegister` type.

In [2]:
ShiftRegister

ShiftRegister = DefineCircuit("ShiftRegister", "I", In(Bits(4)), "O", Out(Bits(4)), "CLK", In(Clock))
inst0 = Register4()
inst1 = Register4()
inst2 = Register4()
inst3 = Register4()
wire(ShiftRegister.I, inst0.I)
wire(ShiftRegister.CLK, inst0.CLK)
wire(inst0.O, inst1.I)
wire(ShiftRegister.CLK, inst1.CLK)
wire(inst1.O, inst2.I)
wire(ShiftRegister.CLK, inst2.CLK)
wire(inst2.O, inst3.I)
wire(ShiftRegister.CLK, inst3.CLK)
wire(inst3.O, ShiftRegister.O)
EndCircuit()

For some separate reason, the visualization is slightly broken (outputs aren't wired up) for a placed circuit / instance (?) like `ShiftRegister()` below.

It's not graphviz-specific, I think: the same issue shows up in firrtl and verilog compile output.

In [3]:
ShiftRegister()

AttributeError: 'ShiftRegister' object has no attribute 'is_definition'

 = ShiftRegister()

### Simple ALU

Another example, taken from the Chisel examples folder.

In [4]:
from functools import reduce

def one_hot_mux(conds, inputs):
    outputs = []
    for cond, inp in zip(conds, inputs):
        outputs.append(inp & uint([cond for _ in range(len(inp))]))
    return reduce(lambda x, y: x | y, outputs)


class SimpleALU(Circuit):
    name = "SimpleALU"
    IO = ["a", In(UInt(4)), "b", In(UInt(4)), "opcode", In(UInt(2)), "out", Out(UInt(4))]
    
    @classmethod
    def definition(io):
        is_op0 = io.opcode == uint(0, n=2)
        is_op1 = io.opcode == uint(1, n=2)
        is_op2 = io.opcode == uint(2, n=2)
        is_op3 = io.opcode == uint(3, n=2)
        op0_out = io.a + io.b
        op1_out = io.a - io.b
        op2_out = io.a
        op3_out = io.b
        wire(io.out, one_hot_mux([is_op0, is_op1, is_op2, is_op3], [op0_out, op1_out, op2_out, op3_out]))

ValueError: bits can only be used on Arrays or Tuples containing bits, not : <class 'list'>

In [ ]:
SimpleALU